# Imports

In [2]:
import pandas as pd
import html
import re
import requests
from bs4 import BeautifulSoup

/tmp/ipykernel_7191/2361605404.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
import time

# Parsing

In [5]:
links_for_parse = ['https://www.zoopicture.ru/tag/zoopark/', 'https://www.zoopicture.ru/category/dead/']

In [ ]:
response = requests.get('https://www.zoopicture.ru/category/dead/page/1/')
print(f'Возможность возвращать json {response.headers["content-type"].strip().startswith("application/json")}' )
print(response.text)
response_text = response.text

In [ ]:
soup = BeautifulSoup(response_text, "html.parser")
#<div class="span main posts" role="main">
href_cut = soup.find_all('h2', {'class':"entry-title"})
for tag in href_cut:
    print(tag.find(href=True).get('href'))

In [22]:
s_t = time.time()
response_page_text = requests.get('https://www.zoopicture.ru/malyj-krolichij-bandikut/').text
soup = BeautifulSoup(response_page_text, "html.parser")
tag_page = soup.find('article').find_all('p')
for tag in tag_page:
    print(tag.text)
print(time.time() - s_t)

Малый кроличий бандикут (лат. Macrotis leucura), некогда известный как яллара, — один из двух видов сумчатых млекопитающих, входящих в род кроличьих бандикутов.

Ялларам, когда-то обитавшим в знойных пустынях в самом сердце австралийского континента, повезло меньше: последний из них был обнаружен в штате Южная Австралия в 1931 году. Затем история этих небольших, похожих на хвостатых кроликов, зверьков обрывается, и сегодня малые кроличьи бандикуты официально считаются вымершим видом.

Впервые малый кроличий бандикут был описан британским зоологом Олдфилдом Томасом в 1887 году. Образцом ученому послужил единственный экземпляр яллары, хранившийся в то время в коллекции Британского Музея. За почти полвека, прошедшие с тех пор и до 1931 года, зоологам попалось в руки всего двенадцать малых кроличьих бандикутов, изучение которых так и не смогло дать ответы на все имевшиеся вопросы.

Известно только, что выраставшие до 30-40 см зверьки предпочитали песчаную и глинистую почву, в которой выкап

In [23]:
soup.find('h1', {'class':'entry-title'}).text

'Малый кроличий бандикут'

In [6]:
def get_urls(url):
    '''
    for exact page parse urls of animals
    '''
    urls = []
    response_text = requests.get(url).text
    soup = BeautifulSoup(response_text, "html.parser")

    href_cut = soup.find_all('h2', {'class':"entry-title"})
    for tag in href_cut:
        urls.append(tag.find(href=True).get('href'))
    return urls

In [25]:
def parse_page(url):
    '''
    for exact animal parse its info
    '''
    page_info = []
    response_page_text = requests.get(url).text
    soup = BeautifulSoup(response_page_text, "html.parser")
    tag_page = soup.find('article').find_all('p')
    for tag in tag_page:
        page_info.append(tag.text)
    title = soup.find('h1', {'class':'entry-title'}).text
    new_data = pd.DataFrame({'name':[title], 'description':[''.join(page_info)], 'url':[url]})
    print(f'parse page {title} ')
    return new_data

In [31]:
parsed_pages = []
errors_dict = {'pages': [], 'site': []}
for link in links_for_parse:
    i = 1
    while requests.get(link + '/page/' + str(i) + '/').status_code == 200:

        # '''!!!'''
        # if i == 3:
        #     break

        try:
            urls = get_urls(link + '/page/' + str(i) + '/')
        except:
            errors_dict['site'].append(link + '/page/' + str(i) + '/')

        i += 1
        for url in urls:
            try:
                parsed_pages.append(parse_page(url))
            except:
                errors_dict['pages'].append(url)

parse page Толстохвостый кускус 
parse page Бело-рыжая летяга 
parse page Красноногий рак-отшельник 
parse page Ринодерма Дарвина 
parse page Рогатый гуан 
parse page Западная лысая ворона 
parse page Гребенчатоголовая жаба 
parse page Розовый морской огурец 
parse page Жуки-нарывники 
parse page Желторотая квакша 
parse page Кузнечик-питбуль 
parse page Восьмиточечный паук-краб 
parse page Коричневый акантозавр 
parse page Причудливая квакша 
parse page Скорпионохвостый паук 
parse page Лопатоносая квакша 
parse page Осмилиды 
parse page Индийская птица-носорог 
parse page Западная американская кобылка 
parse page Носатая рогатая чесночница 
parse page Сиреневогрудая сизоворонка 
parse page Краб-арлекин 
parse page Галактическая сифонофора 
parse page Грета Ото или стеклянная бабочка 
parse page Резиновая змея 
parse page Чешуйчатая райская птица 
parse page Цебидихтис 
parse page Клыкастая квакша 
parse page Скорпион Androctonus amoreuxi 
parse page Геккон Смита 
parse page Обыкновен

In [32]:
zoo_data = pd.concat(parsed_pages)
zoo_data.head()

,name,description,url
0,Толстохвостый кускус,Среди всех сумчатых сонь этот миниатюрный звер...,https://www.zoopicture.ru/cercartetus-nanus/
0,Бело-рыжая летяга,Среди гигантских летающих белок эта обитательн...,https://www.zoopicture.ru/petaurista-alborufus/
0,Красноногий рак-отшельник,Красноногий рак-отшельник (лат. Ciliopagurus s...,https://www.zoopicture.ru/ciliopagurus-strigatus/
0,Ринодерма Дарвина,Честью своего открытия миру эта небольшая южно...,https://www.zoopicture.ru/rhinoderma-darwinii/
0,Рогатый гуан,Прямые потомки самой древней линии семейства к...,https://www.zoopicture.ru/rogatyi-guan/


In [50]:
~zoo_data['name'].str.contains('или')

0     True
0     True
0     True
0     True
0     True
     ...  
0     True
0     True
0     True
0    False
0    False
Name: name, Length: 1469, dtype: bool

In [52]:
zoo_data.loc[(zoo_data['name'].str.split().apply(len) > 4) & (~zoo_data['name'].str.contains('или')) ]

,name,description,url
0,Секрет бабочек из рода Creatonotos,"Среди бабочек-медведиц из рода Creatonotos, об...",https://www.zoopicture.ru/creatonotos/
0,Жабовидные ящерицы вида Phrynosoma hernandesi,У себя на родине они больше известны как рогат...,https://www.zoopicture.ru/phrynosoma-hernandesi/
0,Фиолетовые крабы с острова Палаван,Ученые из Зенкенбергского научно-исследователь...,https://www.zoopicture.ru/fioletovye-kraby-s-o...
0,Белый лев — ожившая легенда,"Всего полвека назад считалось, что белый лев —...",https://www.zoopicture.ru/belyj-lev/
0,Лигры и Тигоны: кто есть кто ?,При скрещивании тигрицы и льва в итоге рождают...,https://www.zoopicture.ru/liger-tigon/
0,"Клещи: все, что нужно знать о клещах",Буквально на днях из живота моего друга вынима...,https://www.zoopicture.ru/vse-chto-nuzhno-znat...
0,Редчайшие фотографии 12 видов вымерших птиц,"Двенадцать видов птиц, о которых пойдет наш ра...",https://www.zoopicture.ru/12-vymershix-ptic/
0,Живы ли слоновые черепахи с Галапагосских остр...,В крови галапагосских черепах учёные обнаружил...,https://www.zoopicture.ru/zhivy-li-slonovye-ch...
0,Кто же истребил гавайских сухопутных крабов?,Появление полинезийских племён тысячу лет наза...,https://www.zoopicture.ru/kto-zhe-istrebil-gav...
0,Шестое массовое вымирание видов началось,Резкое сокращение числа популяций многих живот...,https://www.zoopicture.ru/shestoe-massovoe-vym...


In [60]:
zoo_data = zoo_data.loc[~zoo_data.name.apply(lambda x: any(title in x for title in ['Клещи: все, что нужно знать о клещах', 'Живы ли слоновые черепахи с Галапагосских острровов?', 
 'Кто же истребил гавайских сухопутных крабов?', 'Шестое массовое вымирание видов началось', 
 'Вымершие животные, сфотографированные при жизни', 'Животные, которых может убить глобальное потепление', 'Редчайшие фотографии 12 видов вымерших птиц']))]
#'Редчайшие фотографии 12 видов вымерших птиц' -> Можно Разбить на отдельные статьи при желании 

In [62]:
zoo_data.reset_index(drop=True, inplace=True)
zoo_data.head()

,name,description,url
0,Толстохвостый кускус,Среди всех сумчатых сонь этот миниатюрный звер...,https://www.zoopicture.ru/cercartetus-nanus/
1,Бело-рыжая летяга,Среди гигантских летающих белок эта обитательн...,https://www.zoopicture.ru/petaurista-alborufus/
2,Красноногий рак-отшельник,Красноногий рак-отшельник (лат. Ciliopagurus s...,https://www.zoopicture.ru/ciliopagurus-strigatus/
3,Ринодерма Дарвина,Честью своего открытия миру эта небольшая южно...,https://www.zoopicture.ru/rhinoderma-darwinii/
4,Рогатый гуан,Прямые потомки самой древней линии семейства к...,https://www.zoopicture.ru/rogatyi-guan/


In [ ]:
# zoo_data.to_pickle('zoo_data.xz')

# Data clear

In [64]:
stop_words = ['учен', 'учён', 'снимок', 'фото', 'картин',
              'вымер', 'браконьер', 'млекопит', 'штат',
              'зоолог', 'музе', 'исследовател']

In [63]:
patt_bracketrd = re.compile(r'\([^()]*\)') #Убирает всё, что в круглых скобках "()".
patt_bracketsq = re.compile(r'\[[^\[\]]*\]') #Убирает всё, что в квадратных скобках "[]".

In [66]:
regex_patt_list = [patt_bracketrd, patt_bracketsq]
for patt in regex_patt_list:
    zoo_data.description = zoo_data.description.str.replace(patt, '', regex=True)

In [67]:
zoo_data.head()

,name,description,url
0,Толстохвостый кускус,Среди всех сумчатых сонь этот миниатюрный звер...,https://www.zoopicture.ru/cercartetus-nanus/
1,Бело-рыжая летяга,Среди гигантских летающих белок эта обитательн...,https://www.zoopicture.ru/petaurista-alborufus/
2,Красноногий рак-отшельник,"Красноногий рак-отшельник , обитающий в прибре...",https://www.zoopicture.ru/ciliopagurus-strigatus/
3,Ринодерма Дарвина,Честью своего открытия миру эта небольшая южно...,https://www.zoopicture.ru/rhinoderma-darwinii/
4,Рогатый гуан,Прямые потомки самой древней линии семейства к...,https://www.zoopicture.ru/rogatyi-guan/


In [65]:
def check_me(texts):
    temp_list = []
    texts = re.findall('[^.]+\.', texts)
    for text in texts:
        text = text.lower()
        if len(text) > 4:
            if len(text.split()) > 1:
                if not any(word in text for word in stop_words): 
                    temp_list.append(text)
    return ''.join(temp_list)

In [68]:
zoo_data.description = zoo_data.description.apply(lambda x: check_me(x))

In [73]:
zoo_data.description = zoo_data.description.str.replace(r'[A-Za-z]', '', regex=True)
zoo_data.description = zoo_data.description.str.replace(r'\s{2,}', ' ', regex=True)

In [74]:
list(zoo_data.description.sample(5))

['бурогорлые ленивцы – самый распространенный и многочисленный вид в семействе трехпалых ленивцев. они занимают обширные районы в центральной и южной америке, и только к северу от дождевых лесов амазонии и восточнее риу-негру шансы встретить бурогорлого ленивца практически равны нулю.однако и у себя на родине эти скрытные животные не стремятся к общению. они взбираются на самые высокие деревья и висят неподвижно, стараясь привлекать как можно меньше внимания своих врагов – ягуаров и южноамериканских гарпий. зеленые водоросли, живущие на шершавых волосках их шерсти, делают их еще менее заметными. единственная пища бурогорлых ленивцев – сочные листья тропических растений. чтобы насытиться и получить нужное количество питательных веществ, у ленивцев есть два пути: целый день есть, как это делают жвачные животные, или расходовать энергию так, чтобы съеденного хватило надолго.они предпочли второй вариант. количество листьев, которое усваивается у других животных за час, ленивцы переваривают

In [75]:
zoo_data.shape

(1463, 3)

In [ ]:
zoo_data[['name', 'description']].to_pickle('datas/zoo_data.xz')